# 2. Предобработка данных и получение идентифицирующих признаков

## Часть 1. Предобработка данных

На данном этапе перед нами стоит задача предобработать данные, выгруженные через API Elibrary ранее (см. файл в репозитории `1. Выгрузка данных по статьям`).

**Перед нами стоит несколько задач:**
1. Предобработать данные по русскоязычным фамилиям и инициалам;
2. Предобработать данные по англоязычным фамилиям и инициалам;
3. Предобработать данные по русско- и англоязычным аффилиациям;
4. Провести процедуру присвоения собственных id авторов;

В этой части (`Часть 1`) – нами проводятся первые две процедуры - предобработка данных по русско- и англоязычным фамилиям и инициалам. 


Почему такая необходимость возникла? Дело в том, что изначально мы обнаружили, что в данных достаточно много авторов без id во внутренней системе Elibrary. Этот факт осложняет дальнейшую работу с данными и их анализ – мы не сможем построить сети на основании id авторов, если они не достаточно корректные. Поэтому нами было принято решение подготовить собственные id авторов, основываясь на ФИО и данных об аффилиациях авторов. 


Однако, и на этом этапе возникла проблема – Elibrary не контролирует единообразность записи фамилий, имен и инициалов; туда можно вписать разные знаки препинания; иногда в фамилии вписываются сразу и фамилиии, и инициалы, и так далее.

В этом ноутбуке демонстрируются разные проблемы и способы их решения.

**Получаемые в результате первой чаасти предобработки файлы:**
1. `problem1.xlsx` - строчки с информацией о статьях, для которых отсутствует информация даже о первом(!) авторе (то есть, нет информации об авторах вообще). Необходимо заполнение вручную;
2. `problem2.xlsx` - строчки с информацией о статьях, в фамилиях и инициалах авторов которых встречаются вопросительные знаки. Необходимо заполнение вручную;
3. `problem3.xlsx` - строчки с информацией о статьях, среди авторов которых встречаются авторы с не-русскими фамилиями если в разделе с инициалами у них не указаны иниициалы (то есть, в ячейку с фамилией отнесена и фамилия, и инициалы). Необходимо заполнение вручную.

In [2]:
import pandas as pd # импортируем библиотеку для работы с таблицами
df = pd.read_excel('data.xlsx') # в этом файле находится выгруженная предварительно информация о статьях авторов 
df.drop(axis=1, columns=['Unnamed: 0.1', 'Unnamed: 0', 'Column1'], inplace=True)
df

,id,genreId,author_1_id,author_1_affiliation_id,author_1_ru last name,author_1_ru initials,author_1_ru affiliation,author_1_eng last name,author_1_eng initials,author_1_eng affiliation,...,genre_id,cited,corecited,parent_id,source_id,isFT,isNEW,link,url,ref
0,15485609,4,0.0,"[['7113'], ['7113']]",Кийков,А.В.,['Белгородский государственный технологический...,Kiykov,A.V.,['Belgorod State Technological University name...,...,4,3,1.0,33649315,NaN,1,0,http://elibrary.ru/item.asp?id=15485609,none,"Кийков, А.В. СОСТОЯНИЕ И ТЕНДЕНЦИИ СЕТЕВОГО ВЗ..."
1,15488335,4,675896.0,"['none', 'none']",Салагаев,А.,none,Salagaev,A.,none,...,4,0,0.0,33649638,NaN,1,1,http://elibrary.ru/item.asp?id=15488335,NaN,"Салагаев, А. КУЛЬТУРНАЯ И РЕЛИГИОЗНАЯ СИТУАЦИЯ..."
2,15488853,4,820606.0,"[['340'], 'none']",Семченко,И.В.,['Белгородский государственный университет'],Semchenko,I.V.,none,...,4,0,0.0,33649676,NaN,1,1,http://elibrary.ru/item.asp?id=15488853,NaN,"Семченко, И.В. ПРОБЛЕМЫ СОЦИАЛЬНОЙ АДАПТАЦИИ Г..."
3,15500786,4,77952.0,"[['210'], 'none']",Хагуров,Т.А.,['Кубанский государственный университет'],Hagurov,T.A.,none,...,4,0,0.0,33651468,NaN,1,1,http://elibrary.ru/item.asp?id=15500786,NaN,"Хагуров, Т.А. Современная культура и воспитани..."
4,15501422,4,746025.0,"[['2541'], 'none']",Елишев,С.О.,['МГУ им. М.В. Ломоносова?'],Elishev,S.O.,none,...,4,22,3.0,33651490,NaN,0,1,http://elibrary.ru/item.asp?id=15501422,NaN,"Елишев, С.О. Формирование ценностных ориентаци..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75227,48109813,4,725411.0,"['none', ['211']]",none,none,none,Shayakhmetova,R.R.,['Bashkiria State University'],...,4,0,0.0,48109812,17.0,1,1,http://elibrary.ru/item.asp?id=48109813,https://revista.fct.unesp.br/index.php/Nuances...,VALUES OF SELF-PRESERVING BEHAVIOR IN THE SYST...
75228,47812338,4,702706.0,"['none', ['190']]",none,none,none,Kretser,I.Yu.,['Saint-Petersburg State University'],...,4,0,0.0,46863661,17.0,0,1,http://elibrary.ru/item.asp?id=47812338,https://polarjournal.org/2020/03/18/migration-...,"Kretser, I.Y. Domestic Economies: Women, Work,..."
75229,47325063,4,1130043.0,"[['1052'], 'none']",Касаткина,М.И,['Калужский государственный университет им. К....,Kasatkina,M.I,none,...,4,8,0.0,47325062,17.0,0,1,http://elibrary.ru/item.asp?id=47325063,NaN,"Касаткина, М.И. Организация работы с пожилыми ..."
75230,48032031,4,910803.0,"['none', 'none']",Дыжова,А.А.,none,Dyzhova,A.A.,none,...,4,0,0.0,48032030,17.0,0,1,http://elibrary.ru/item.asp?id=48032031,NaN,"Дыжова, А.А. Особенности правовой социализации..."


### 0. Создание пользовательских функций для работы с данными

Перед тем, как работать дальше, я создаю несколько функций, проверяющих язык написания текста и функций, транслитерирующих текст. Это нужно, тк в редких случаях в ячейке с русскоязычными инцииалами могут быть записаны англоязычные, в тексте с англоязычными - русскоязычные, и так далее.

In [7]:
def is_in_english(text, alphabet=set('abcdefghijklmnopqrstuvwxyz')):
    return not alphabet.isdisjoint(text.lower())
print(is_in_english('test')) # True
print(is_in_english('тест')) # False
print(is_in_english('123Ы')) # False
print(is_in_english('Фаmilia')) # есть английские символы, True
print(is_in_english(''))     # False

True
False
False
True
False


In [9]:
def is_in_russian(text, alphabet=set('абвгдеёжзийклмнопрстуфхцчшщъыьэюя')):
    return not alphabet.isdisjoint(text.lower())

print(is_in_russian('test')) # False
print(is_in_russian('тест')) # True
print(is_in_russian('123Ы')) # True
print(is_in_russian('Фаmilia')) # есть русские символы, True
print(is_in_russian(''))     # False

False
True
True
True
False


In [17]:
# использую готовую библиотеку транслитерации
#!pip install transliterate
from transliterate import translit
text = 'Lorem ipsum dolor sit amet'
ru_text = translit(text, 'ru') # text мы транслитерируем на русский
ru_text

'Лорем ипсум долор сит амет'

In [18]:
def transliterate_to_russian(text, capitalize=True):
    ru_text = translit(text, 'ru').lower() # text мы транслитерируем на русский
    ru_text = ru_text.replace('кх', 'х').replace('ыа', 'я').replace('ые', 'ь').replace('ыу', 'ю') # меняем 
    ru_text = ru_text.replace('ьва', 'ева')
    if capitalize == True:
        ru_text = ru_text.capitalize()
    return ru_text

In [19]:
def transliterate_to_english(text, capitalize=True):
    eng_text = translit(text, 'ru', reversed=True).lower() # text мы транслитерируем на английский
    if capitalize == True:
        eng_text = eng_text.capitalize()
    return eng_text

## Часть 1. Содержание предобработки:


* **Шаг 0.** Ранее я поняла, что у нас двумя способами маркировались отсутствующие данные: как `np.nan` (фильтруется через `.isnull() == True`) и через `none`.  `none` встречается чаще, просто отсутствующие данные встречаются реже. Поэтому **выше была проведена замена отсутствующих значений (`np.nan`)на `none`**;


* **Шаг 1.** Нужно во всех столбцах `author_{i}_eng initials` заменить n. на none; если не с n. - то просто привести к виду `X.X.`;


* **Шаг 2.** Заменить в столбцах `author_{i}_ru initials` строки где с малой строки: если начинаются на n. - на none; если не с n. - то просто привести к виду N.N. **Далее произвести транслитерацию** - `author_{i}_ru initials` в `author_{i}_eng initials` для таких наблюдений;


* **Шаг 3**. В тех инициалах, где по столбцу `author_{i}_ru_initials` присутствует русская буква `X` и на ее месте в столбце `author_{i}_eng_initials` находится английская `k`, следует заменить ее на `h`;


* **Шаг 4**. Исследовать проблемные фамилии (в которых присутствуют точки) по всем столбцам `author_{i}_ru last name` и `author_{i}eng last name`: взять из фамилий инициалы и поставить их на место инициалов `author_{i}_{j} initials`, а сами инициалы удалить из фамилий `author_{i}_{j} last name`. **Далее произвести транслитерацию** - `author_{i}_ru initials` в `author_{i}_eng initials` для таких наблюдений;


* **Шаг 5**. Поработать с проблемными инициалами `author_{i}_ru initials` там где более двух точек в инициалах: делим строку по точкам, используем .strip(), берем только те значения, длина которых меньше двух(Так, Н.Г.-К. станет Н.Г.-К; а вот Р.А. Роман Александрович преобразуется просто в Р.А.). Транслитерировать инициалы в `author_{i}_eng initials`;


* **Шаг 6**. Транслитерировать с английского на русский в столбцах `author_{i}_ru last name` и `author_{i}_ru initials`, где указан английский, а не русский текст.

### Шаг 0.  Замена `np.nan` на строку `none`

In [14]:
import numpy as np
df = df.replace(np.nan, 'none') # замена np.nan на 'none'

### Шаг 1. Транслитерация с английского на русский в столбцах: обработка `author_{i}_eng initials`

В некоторых столбцах с пометкой `ru` на самом деле содержатся написанные на английском фамилии. Надо перевести все `last name` и `initials` в столбцах со всеми `8` авторами с пометкой `ru`.

Что будем делать? Во всех столбцах `author_{i}_eng initials` заменю `n.` на `none` (есть предположение, что в рамках небольшой нашей собственной предварительной обработки значение `none` было приведено к `n.`), и приведу к прописным буквам.

In [21]:
# пример проблемы - в столбце с русскими фамилиями стоят фамилии на английиском
for i in df['author_1_ru last name'][:25000]:
    if is_in_russian(i) != True and i != 'none':
        print(i)

Kahaeba
Tatarko
Kulyasov
Kulyasov
Sadykova


In [66]:
for i in ['Попова', 'Колосова', 'Львов']:
    print(transliterate_to_english(i)) # пример работы функции

Popova
Kolosova
L'vov


In [20]:
# пример транслитерации с английского на русский
for i in df[df['author_1_ru last name'] == 'none']['author_1_eng last name'].tolist()[50:60]:
    try:
        print(f'Фамилия: {i}, перевод с помощью translator: {transliterate_to_russian(i)}')
    except:
        print(f'Не получилось перевести {i}')

Фамилия: Vasiliev, перевод с помощью translator: Василиев
Фамилия: Romashkina, перевод с помощью translator: Ромашкина
Фамилия: Tuhvatullina, перевод с помощью translator: Тухватуллина
Фамилия: Turdubayeva, перевод с помощью translator: Турдубаева
Фамилия: Lvov, перевод с помощью translator: Лвов
Фамилия: Kolosova, перевод с помощью translator: Колосова
Фамилия: Dolomatov, перевод с помощью translator: Доломатов
Фамилия: Klimyuk, перевод с помощью translator: Климюк
Фамилия: Salova, перевод с помощью translator: Салова
Фамилия: Kornilova, перевод с помощью translator: Корнилова


In [22]:
def change_value(x):
    if x == 'n.' or x == 'none':
        return 'none'
    else:
        return x.upper()
    
for i in range(1, 8+1):
    df[f'author_{i}_eng initials'] = df[f'author_{i}_eng initials'].apply(lambda x: change_value(x))

### Шаг 2 – обработка `author_{i}_ru initials` и `author_{i}_eng initials`

Заменить в столбцах `author_{i}_ru initials` строки где с малой строки: если начинаются на n. - на none; если не с n. - то просто привести к виду `X.X`. **Далее произвести транслитерацию** - `author_{i}_ru initials` в `author_{i}_eng initials` для таких наблюдений;

In [139]:
# Демонстрация проблемы
for i in df['author_1_ru initials']:
    if i.islower() == True and i != 'none':
        print(i)

n.
n.
n.
n.
n.
n.
n.
n.
n.
n.
n.
n.
к.соц.н.
n.
n.
n.
n.
n.
n.
n.
n.
n.
n.
n.
n.
n.
n.
n.
n.
n.
n.
n.
n.
n.
n.
n.
n.
n.
n.
n.
n.
n.
n.
n.
n.
n.
n.
n.
n.
n.
n.
n.
т.к.
n.
д.с.
n.
е.ю.
n.
n.
n.
n.
n.
n.


In [140]:
for i in range(1, 8+1):
    col_index = i
    
    for j in range(df.shape[0]):
        
        row_index = j
        current_ru_initials = df.loc[j, f'author_{i}_ru initials']
        # если наши инициалы написаны с маленькой буквы
        if current_ru_initials.islower() == True:
            # если у нас инициалы n. или none
            if current_ru_initials == 'n.' or current_ru_initials == 'none' or current_ru_initials == 'к.соц.н':
                df.loc[j, f'author_{i}_ru initials'] = 'none' # присваиваем none
                df.loc[j, f'author_{i}_eng initials'] = 'none' # присваиваем none
            else: # если не none, просто с маленькой буквы
                df.loc[j, f'author_{i}_ru initials'] = current_ru_initials.upper() # по-русскки - поднимаем вверх
                df.loc[j, f'author_{i}_eng initials'] = transliterate_to_english(current_ru_initials.upper()) # транслитерируем на английский

In [141]:
# Демонстрация проблемы: РЕШЕНО. 
for i in df['author_1_ru initials']:
    if i.islower() == True and i != 'none':
        print(i) 

Теперь у нас нет проблемных значений с маленькой буквы в колонках с русскими инициалами. 

### Шаг 3  – работа с русской Х в  `author_{i}_ru_initials` и  `author_{i}_eng_initials`
В тех инициалах, где по столбцу `author_{i}_ru_initials` присутствует русская буква `X` и на ее месте в столбце `author_{i}_eng_initials` находится английская `k`, следует заменить ее на `h`. Это возникает, вероятно, из-за специфики транслитерации.

In [142]:
for i in range(1, 8+1):
    index_col = i # идем по 8 столбцам с авторами
    for j in range(df.shape[0]):
        index_row = j  # смотрим на конкретную строчку
        current_russian_initials = df.loc[index_row, f'author_{i}_ru initials']
        if 'х' in current_russian_initials.lower(): # если в инициалах есть х
            # то в английской транслитерации заменяем k на h (х транслитерируется как кх)
            df.loc[index_row, f'author_{i}_eng initials'] = df.loc[index_row, f'author_{i}_eng initials'].replace('k', 'h').upper()

### Шаг 4 – предобработка фамилий `author_{i}_ru last name` и `author_{i}_eng last name`
Исследовать проблемные фамилии (в которых присутствуют точки) по всем столбцам `author_{i}_ru last name` и `author_{i}eng last name`: взять из фамилий инициалы и поставить их на место инициалов `author_{i}_{j} initials`, а сами инициалы удалить из фамилий `author_{i}_{j} last name`. **Далее произвести транслитерацию** - `author_{i}_ru initials` в `author_{i}_eng initials` для таких наблюдений;

In [75]:
# Демонстрация проблемы - иногда в фамилиях (last name) записаны еще и инициалы
for i in df['author_1_ru last name']:
    if '.' in i:
        print(i)

Савченко Е.
Шилова.
Крикунова Ю.А.
Т.И. Морозова
Б.Усманов
Шафранов-Куцев Г.Ф.
В.Н. Круглиевский
Сотникова Н.Н.
Ханаху Р.А
Ин Д.В
Воронежский И.С.
Двирник. Ю.С
Попова.В.В
Лук. П.А


In [143]:
# Пример - как это будет работать; разобраны разные сценарии
# case 1 - "Иванов И.И" в author_{i}_ru last name
# case 2 - "Иванов." в author_{i}_ru last name
# case 3 - "И.И.Иванов." в author_{i}_ru last name

for i in df['author_1_ru last name']:
    if '.' in i:
        if ' ' in i:
            last_name_list = i.split()
            last_name_list = sorted(last_name_list, key=lambda x: x.isupper()) # сортируем по в вверхнем регистре или нет
            last_name = last_name_list[0].strip('.') # корректная фамилия
            initials = '.'.join(last_name_list[1:]).upper() # корректные инициалы
            print('case 1')
            print(f'was {i}, became {last_name},\nwill transfer to initials {initials}', end='\n\n')
            
        else:
            last_name_list = i.split('.')
            last_name_list = sorted(last_name_list, key=lambda x: x.isupper()) # сортируем: сперва будут значения которые не полностью капслоком, потом - такие
            if len(last_name_list) == 2 and last_name_list[1] == '': # если у нас там только фамилия (пример: Шилова. -> Шилова)
                new_last_name = last_name_list[0]
                print('case 2')
                print(f'was {i}, became {new_last_name}', end='\n\n')
            else:
                new_last_name = last_name_list[0].capitalize()
                new_initials = '.'.join(last_name_list[1:]).upper() + '.'
                print('case3')
                print(f'was {i}, became {new_last_name},\nwill transfer to initials {new_initials}', end='\n\n')
                    

case 1
was Савченко Е., became Савченко,
will transfer to initials Е.

case 2
was Шилова., became Шилова

case 1
was Крикунова Ю.А., became Крикунова,
will transfer to initials Ю.А.

case 1
was Т.И. Морозова, became Морозова,
will transfer to initials Т.И.

case3
was Б.Усманов, became Усманов,
will transfer to initials Б.

case 1
was Шафранов-Куцев Г.Ф., became Шафранов-Куцев,
will transfer to initials Г.Ф.

case 1
was В.Н. Круглиевский, became Круглиевский,
will transfer to initials В.Н.

case 1
was Сотникова Н.Н., became Сотникова,
will transfer to initials Н.Н.

case 1
was Ханаху Р.А, became Ханаху,
will transfer to initials Р.А

case 1
was Ин Д.В, became Ин,
will transfer to initials Д.В

case 1
was Воронежский И.С., became Воронежский,
will transfer to initials И.С.

case 1
was Двирник. Ю.С, became Двирник,
will transfer to initials Ю.С

case3
was Попова.В.В, became Попова,
will transfer to initials В.В.

case 1
was Лук. П.А, became Лук,
will transfer to initials П.А



In [144]:
for i in range(1, 8+1):
    index_col = i
    
    for j in range(df.shape[0]):
        index_row = j
        current_last_name_ru = df.loc[index_row, f'author_{index_col}_ru last name']
        if '.' in current_last_name_ru:
            
            # если у нас в фамилии и имени есть пробелы
            if ' ' in current_last_name_ru:
                last_name_list = current_last_name_ru.split()
                last_name_list = sorted(last_name_list, key=lambda x: x.isupper()) # сортируем по в вверхнем регистре или нет
                
                last_name = last_name_list[0].strip('.') # корректная фамилия
                initials = '.'.join(last_name_list[1:]).upper() # корректные инициалы
                
                # присваиваем значения: выносим из фамилии инициалы в свой столбец; у себя оставляем только фамилию
                df.loc[index_row, f'author_{index_col}_ru last name'] = last_name
                df.loc[index_row, f'author_{index_col}_ru initials'] = initials
                # транслитерируем с русского на англиийский
                df.loc[index_row, f'author_{index_col}_eng last name'] = transliterate_to_english(last_name)
                df.loc[index_row, f'author_{index_col}_eng last name'] = transliterate_to_english(initials).upper()
            else:
                last_name_list = current_last_name_ru.split('.') # разбиваем по точке
                last_name_list = sorted(last_name_list, key=lambda x: x.isupper()) # сортируем: сперва будут значения которые не полностью капслоком, потом - такие
                
                if len(last_name_list) == 2 and last_name_list[1] == '': # если у нас там только фамилия (пример: Шилова. -> Шилова)
                    new_last_name = last_name_list[0]
                    # просто обновляем фамилию и транслитерируем ее на английский
                    df.loc[index_row, f'author_{index_col}_ru last name'] = new_last_name
                    df.loc[index_row, f'author_{index_col}_eng last name'] = transliterate_to_english(last_name)
                else:
                    # а если у нас там и фамилия, и иницииалы (пример: Попова.В.В -> [Попова,В,В])
                    new_last_name = last_name_list[0].capitalize()
                    new_initials = '.'.join(last_name_list[1:]).upper() + '.'
                    
                    # теперь выносим из ячейки инициалы в русские инициалы, оставляем только фамилию
                    df.loc[index_row, f'author_{index_col}_ru last name'] = new_last_name
                    df.loc[index_row, f'author_{index_col}_ru initials'] = new_initials
                    # потом все это дело транслитерируем на английиский
                    df.loc[index_row, f'author_{index_col}_eng last name'] = transliterate_to_english(new_last_name)
                    df.loc[index_row, f'author_{index_col}_eng last name'] = transliterate_to_english(new_initials).upper()
                    

In [145]:
# Проблема - решена! - ничего не печатается
for i in df['author_1_ru last name']:
    if '.' in i:
        print(i)

### Шаг 5 - работа с инициалами в `author_{i}_ru initials`, где указано больше двух точек

Задача на этом шаге – поработать с проблемными инициалами `author_{i}_ru initials` там где более двух точек в инициалах: делим строку по точкам, используем метод строк `.strip()`, берем только те значения, длина которых меньше двух. Для примера: `Н.Г.-К.` останется `Н.Г.-К`, но `Р.А. Роман Александрович` преобразуется просто в `Р.А.`. После этих преобразований мы транслитерируем инициалы в `author_{i}_eng initials`.

In [23]:
# демонстрация проблемы - иногда подозрительно много точек 
for i in df['author_1_ru initials']:
    if i.count('.') > 2:
        print(i)

к.соц.н.
Н.Г.-К.
М.С.-Г.
З.Х.-М.
Н. Г.-К.
О.-В.В.
Э.А.-О.
Б.М.-Г.
И.И. Биктагирова М.К.
Н. С.-Х.
Н. С.-Х.
М.-П.Б.
В. М.I.1О8I.
С.-А. И.
Е.А. Савостьянов В.М.
А.А.Х.
З. Х.-М.
Р.А.-А.


In [148]:
# демонстрация проблемы и работы
for i in df['author_1_ru initials']:
    if len(i) > 4:
        # пример1 - "Р. Д."
        if len(i.strip(',')) in [4, 5] and i.strip(',')[-1] == '.' and i[1] == '.' and i[2] == ' ':
            new_initials = i.replace(' ', '').strip(',')
            print(i)
            print(new_initials)
            print()
        # пример 2: Огребор О.Д.
        elif len(i.split()) == 2 and i.count('.') > 1 and '-' not in i:
            print(i)
            print('Нельзя преобразовать, нужно игнорировать')
            print()
        
        # пример 3: Владимир И.
        elif len(i.split()) == 2 and '.' in i and len(i) != 5:
            print(i)
            mylist = i.strip('.').split()
            mylist = [i.replace('.', '') for i in mylist]
            print(mylist)
            new_initials = []
            for z in mylist:
                # пример: Дмитрий
                if '(' not in z and '-' not in z and '?' not in z:
                    new = z[0].upper()
                    new_initials.append(new)
                # пример: Азер-кызы 
                elif '-' in z:
                    first_symbol = z[0].upper()
                    second_symbol = z[z.index('-')+1].upper()
                    new = f'{first_symbol}.-{second_symbol}.'
                    new_initials.append(new)
                    
                elif '(' in z or '?' in z:
                    new_initials.append(z) # просто добавляем к новым инициалам
            print(new_initials)
            new_initials = '.'.join(new_initials)
            if new_initials[-1] != '.':
                new_initials += '.'
            print(new_initials)
            print()
                    

Т. Азер-кызы
['Т', 'Азер-кызы']
['Т', 'А.-К.']
Т.А.-К.

Екатерина Михайловна.
['Екатерина', 'Михайловна']
['Е', 'М']
Е.М.

Дмитрий Сергеевич.
['Дмитрий', 'Сергеевич']
['Д', 'С']
Д.С.

Вл. А.
Нельзя преобразовать, нужно игнорировать

Уэсли У.
['Уэсли', 'У']
['У', 'У']
У.У.

Александр Ф.
['Александр', 'Ф']
['А', 'Ф']
А.Ф.

Данило Ж.
['Данило', 'Ж']
['Д', 'Ж']
Д.Ж.

Джеймс Б.
['Джеймс', 'Б']
['Д', 'Б']
Д.Б.

Пол Ф.
['Пол', 'Ф']
['П', 'Ф']
П.Ф.

Айгуль Ф.
['Айгуль', 'Ф']
['А', 'Ф']
А.Ф.

Вячеслав Вениаминович.
['Вячеслав', 'Вениаминович']
['В', 'В']
В.В.

Андрей Н.
['Андрей', 'Н']
['А', 'Н']
А.Н.

Го Ун.
['Го', 'Ун']
['Г', 'У']
Г.У.

З. Х-М
['З', 'Х-М']
['З', 'Х.-М.']
З.Х.-М.

Н. Г.-К.
['Н', 'Г-К']
['Н', 'Г.-К.']
Н.Г.-К.

Веков А.И.
Нельзя преобразовать, нужно игнорировать

Джон Дж.
['Джон', 'Дж']
['Д', 'Д']
Д.Д.

Дмитрий Юрьевич.
['Дмитрий', 'Юрьевич']
['Д', 'Ю']
Д.Ю.

Сергей Александрович.
['Сергей', 'Александрович']
['С', 'А']
С.А.

Екатерина В.ячеславовна
['Екатерина', 'Вячеславовна']


In [149]:
for i in range(1, 8+1):
    index_col = i
    
    for j in range(df.shape[0]):
        current_initials = df.loc[j, f'author_{index_col}_ru initials']
        
        # случай 1: инициалы "Д. Р.", нужно просто убрать пробел между
        if len(current_initials.strip(',')) in [4, 5] and current_initials.strip(',')[-1] == '.' and current_initials[1] == '.' and current_initials[2] == ' ':
            new_initials = current_initials.replace(' ', '').strip(',')
            
            # теперь! меняем инициалы нормальные; русские инициалы транслитерируем на английский
            df.loc[j, f'author_{index_col}_ru initials'] = new_initials
            df.loc[j, f'author_{index_col}_eng initials'] = transliterate_to_english(new_initials).upper()

        # пример 2: Огребор О.Д.
        elif len(current_initials.split()) == 2 and current_initials.count('.') > 1 and '-' not in current_initials:
            # преобразовать невозможно, пропускаем
            pass
        
        # пример 3: "Владимир И." или "Сергей Александрович."
        elif len(current_initials.split()) == 2 and '.' in current_initials and len(current_initials) != 5:
            # делаем список
            mylist = current_initials.strip('.').split()
            mylist = [k.replace('.', '') for k in mylist] # удаляем точки
            new_initials = []
            
            # идем по тому что было в изначальной строке; пример: ['Александр', 'Сергеевич']
            for z in mylist:
                # пример: Дмитрий
                if '(' not in z and '-' not in z and '?' not in z:
                    new = z[0].upper() # на выходе: Д
                    new_initials.append(new) 
                # пример: Азер-кызы 
                elif '-' in z:
                    first_symbol = z[0].upper()
                    second_symbol = z[z.index('-')+1].upper()
                    new = f'{first_symbol}.-{second_symbol}' # результат - 'Т.-К'
                    new_initials.append(new)
                    
                elif '(' in z or '?' in z:
                    new_initials.append(z) # просто добавляем к новым инициалам, пример: (де)

            new_initials = '.'.join(new_initials) # объединяем с точками, теперь это строка а не список
            if new_initials[-1] != '.':
                new_initials += '.'
            # теперь! меняем инициалы нормальные; русские инициалы транслитерируем на английский
            df.loc[j, f'author_{index_col}_ru initials'] = new_initials
            df.loc[j, f'author_{index_col}_eng initials'] = transliterate_to_english(new_initials).upper()

In [151]:
# демонстрация проблемы и работы: часть проблем ушла, остались вот такие случаи
for i in df['author_1_ru initials']:
    if len(i) > 4 and '-' not in i:
        print(i)

Оксана Геннадьевна Г.
Вл. А.
В.Дж.
Вл.В.
Р.А. Роман Александрович
Вл.А.
О.М.Доктор
Вл.А.
Вл.А.
Веков А.И.
Оксана.
Хава Мохаммад М.
(де).В.
Одриосола М.С.
Игнатий (Н.Д. Киут)
Шади А.А.
И.И. Биктагирова М.К.
Алаба.с
Хасан М.А.
E.Yu.
(де).В.
В. М.I.1О8I.
I.Yu.
T.Yu.
Куюденко А. С.
? С.С.
Е.А. Савостьянов В.М.
А.А.Х.
Р. Мл.
Дж.Б.
Огбебор О.Д.
Ya.V.


### Шаг 6. Транслитерация с английского на русский в столбцах `author_{i}_ru last name` и `author_{i}_ru initials`, где указан английский, а не русский текст

В наших данных естьь строчки в столбцах с меткой `ru`, где на самом деле содержатся англоязычные строки. Их нужно транслитерировать на русский; изначальное значение строки перенести в английский `author_{i}_eng last name`, а транслитерацию оставить в `author_{i}_ru last name`

In [153]:
# демонстрация проблемы: на месте РУССКИХ фамилий находятся АНГЛИЙСКИЕ
for i in df['author_1_ru last name']:
    if is_in_russian(i) == False and i != 'none':
        print(i)

Kahaeba
Tatarko
Kulyasov
Kulyasov
Sadykova
Goldberg-Altyntsev
Aksenov
Cherkasova
Chahkiev
Korol
Andrienko
Kasymov
Fomicheva
Kutsov
Skorohodova
Tanatova
Rohmistrov
Chankova
Saari
Shimanskaya
Kirilina
Marshak
Karstein
Mikhoparov
Steklov
Tanatova
Fetisova
Kravchenko
Igebaeva
Ediltrudis
Golovina
Zakharova
Cherezov
Bryazgina
Galindabaeva
Mukomel'
Shnyrenkov
Anufriev
Anufriev
Pryadcko
Yanitsky
Arkhipova
Ermolaeva
Mannila
Shimanovskaya


In [157]:
for i in range(1, 8+1):
    df[f'author_{i}_ru last name'] = df[f'author_{i}_ru last name'].apply(lambda x: transliterate_to_russian(x) if is_in_russian(x) == False and x != 'none' else x)

In [158]:
# демонстрация проблемы - проблемы больше нет
for i in df['author_1_ru last name']:
    if is_in_russian(i) == False and i != 'none':
        print(i)

In [159]:
# демонстрация проблемы - РУССКИЕ фамилии там, где должны быть АНГЛИЙСКИЕ
for i in df['author_1_eng last name']:
    if is_in_english(i) == False and i != 'none':
        print(i)

Фролова
Федорова
Андриенко
Тау


In [161]:
for i in range(1, 8+1):
    df[f'author_{i}_eng last name'] = df[f'author_{i}_eng last name'].apply(lambda x: transliterate_to_english(x) if is_in_english(x) == False and x != 'none' else x)

In [162]:
# демонстрация проблемы - РУССКИЕ фамилии на месте АНГЛИЙСКИХ - решено!
for i in df['author_1_eng last name']:
    if is_in_english(i) == False and i != 'none':
        print(i)

### Часть 1. Проблемы,  требующие ручной обработки

Во-первых, я обнаружила, что есть **Отсутствующие первые авторы у 35 статей**.
Это статьи с отсутствующими данными по первым авторам находятся по фильтрации `df[df['author_1_eng last name'] == 'none']`. Ручная проверка показывает, что у некоторых авторы все-таки есть имена и данные о них множно получить; пример – первая [статья](https://elibrary.ru/item.asp?id=22649492) из этого списка; по ссылке находятся и данные о статье, и данные об [авторе](https://elibrary.ru/author_profile.asp?authorid=757767). Предлагаемое решение - досбор данных вручную (благо, в нашем случае это 0.05% от всех наших данных - 35 cтатей).

In [209]:
# Отсутствующие первые авторы у 35 статей 
df[df['author_1_eng last name'] == 'none']

,id,genreId,author_1_id,author_1_affiliation_id,author_1_ru last name,author_1_ru initials,author_1_ru affiliation,author_1_eng last name,author_1_eng initials,author_1_eng affiliation,...,genre_id,cited,corecited,parent_id,source_id,isFT,isNEW,link,url,ref
18382,22649492,4,none,"['none', 'none']",none,none,none,none,none,none,...,4,0,0.0,34040276,none,1,0,http://elibrary.ru/item.asp?id=22649492,none,"Ковтун, Н.М. ДЕСТРУКТИВНІСТЬ ЯК СКЛАДОВА СОЦІА..."
25295,21322840,4,none,"['none', 'none']",none,none,none,none,n.,none,...,4,0,0.0,33948873,none,1,0,http://elibrary.ru/item.asp?id=21322840,none,"Грібер, Ю.О. КОЛЬОРОВА ДИНАМІКА МІСЬКОГО ПРОСТ..."
31014,24710185,4,none,"['none', 'none']",none,none,none,none,n.,none,...,4,0,0.0,34171258,none,1,0,http://elibrary.ru/item.asp?id=24710185,none,"Скаленко, О.К. ГЛОБАЛЬНІ ВЗАЄМОЗВ’ЯЗКИ І ВСЕЗА..."
32820,21598217,4,none,"['none', 'none']",none,none,none,none,n.,none,...,4,0,0.0,33963364,none,1,0,http://elibrary.ru/item.asp?id=21598217,none,"ФАУЗЕР, В.В. ВЛИЯНИЕ МИГРАЦИИ НА ЭТНОДЕМОГРАФИ..."
34408,29454634,4,none,"['none', 'none']",none,none,none,none,n.,none,...,4,1,0.0,34488476,none,1,0,http://elibrary.ru/item.asp?id=29454634,none,"Hern?ndez, O. Particularidades del desarrollo ..."
46775,32383601,4,none,"['none', 'none']",none,none,none,none,n.,none,...,4,0,0.0,34830439,none,1,0,http://elibrary.ru/item.asp?id=32383601,none,"Профессия""тьютор""всовременном российском социа..."
56231,37288426,4,none,"['none', 'none']",none,none,none,none,n.,none,...,4,0,0.0,37288376,none,1,1,http://elibrary.ru/item.asp?id=37288426,none,MANAGEMENT OF NATIONAL RELATIONS AS THE IMPLEM...
57347,39822976,4,none,"['none', 'none']",none,none,none,none,n.,none,...,4,9,0.0,39822952,none,1,1,http://elibrary.ru/item.asp?id=39822976,http://ras.jes.su/socis/s013216250006181-2-1,Образ жизни и отношение к здоровью российских ...
57399,41138509,4,none,"['none', 'none']",none,none,none,none,n.,none,...,4,0,0.0,41138501,none,1,1,http://elibrary.ru/item.asp?id=41138509,none,"Skulmovskaya, L.G. L'influence Mutuelle Des Cu..."
58866,41238908,4,none,"['none', 'none']",none,none,none,none,n.,none,...,4,0,0.0,41238907,none,0,0,http://elibrary.ru/item.asp?id=41238908,none,"Golovina, Е.V. PROBL?MES SOCIO-CULTURELS DE LA..."


In [164]:
problem1 = df[df['author_1_eng last name'] == 'none'] # создаем отдельный датафрейм
problem1.to_excel('problem1.xlsx', index=None) # сохраняем проблемные строчки в отдельный датасет

**Во-вторых**, нами было обнаружено, что иногда в фамилиях оказываются вопросы. Нет какой-то универсальной логики по которым они там оказываются, как видно по таблице ниже, поэтому они требуют ручной обработки. Встречаемость – 0.01% в наших данных (в абсолютном значении - 11 случаев). 

In [26]:
round(11 / df.shape[0] * 100, 2)

0.01

In [172]:
problem2_ids = []
for j in range(df.shape[0]):
    for i in range(1, 8+1):
        author_eng_last_name = df.loc[j, f'author_{i}_eng last name']
        author_ru_last_name = df.loc[j, f'author_{i}_ru last name']
        if '?' in author_eng_last_name or '?' in author_ru_last_name:
            problem2_ids.append(df.loc[j, 'id'])

In [173]:
len(problem2_ids)  # всего

11

In [175]:
problem2 = df[df['id'].isin(problem2_ids)]
problem2

,id,genreId,author_1_id,author_1_affiliation_id,author_1_ru last name,author_1_ru initials,author_1_ru affiliation,author_1_eng last name,author_1_eng initials,author_1_eng affiliation,...,genre_id,cited,corecited,parent_id,source_id,isFT,isNEW,link,url,ref
4165,17775662,4,641147.0,"[['820'], 'none']",П?розов,В.А.,['Пермский государственный педагогический унив...,P?rozov,V.A.,none,...,4,0,0.0,33737245,none,1,0,http://elibrary.ru/item.asp?id=17775662,none,"П?розов, В.А. Образ интеллигенции российского ..."
8406,15628783,4,0.0,"[['633'], 'none']",Воробь?ва,Т.Ю.,['Южно-Российский государственный технический ...,Vorob'?va,T.JU.,none,...,4,0,0.0,33657214,none,1,1,http://elibrary.ru/item.asp?id=15628783,none,"Воробь?ва, Т.Ю. ЦВЕТОВАЯ ДИНАМИКА МОДЫ ИЛИ МОД..."
29850,21723566,4,368797.0,"[['1025'], 'none']",Литвинова,Л.В.,['Филиал государственного бюджетного образоват...,Litvinova,L.V.,none,...,4,0,0.0,33970688,none,0,1,http://elibrary.ru/item.asp?id=21723566,none,"Литвинова, Л.В. НЕПРАВИЛЬНОЕ ПИТАНИЕ, КАК ИСТО..."
31501,26460079,4,572584.0,"[['634'], ['']]",Шаов,А.А.,"['Адыгейский государственный университет, г. М...",Shaov,A.A.,"['Adygeya State University, Maykop, Russia']",...,4,8,0.0,34254698,none,1,0,http://elibrary.ru/item.asp?id=26460079,none,"ШАОВ, А.А. НАЦИОНАЛЬНЫЕ ИНТЕРЕСЫ В КОНТЕКСТЕ Э..."
32049,21723557,4,0.0,"[['1025'], 'none']",Куркина,Е.И.,['Филиал государственного бюджетного образоват...,Kurkina,E.I.,none,...,4,0,0.0,33970688,none,0,1,http://elibrary.ru/item.asp?id=21723557,none,"Куркина, Е.И. АНАЛИЗ РЕАЛИЗАЦИИ ДОЛГОСРОЧНОЙ Ц..."
32050,21723558,4,0.0,"[['1025'], 'none']",Гаврилова,К.В.,['Филиал государственного бюджетного образоват...,Gavrilova,K.V.,none,...,4,0,0.0,33970688,none,0,1,http://elibrary.ru/item.asp?id=21723558,none,"Гаврилова, К.В. СОЦИАЛЬНАЯ ЗАЩИТА ДЕТЕЙ-ИНВАЛИ..."
32051,21723571,4,0.0,"[['1025'], 'none']",Гуреева,А.А.,['Филиал государственного бюджетного образоват...,Gureeva,A.A.,none,...,4,0,0.0,33970688,none,0,1,http://elibrary.ru/item.asp?id=21723571,none,"Гуреева, А.А. СПОСОБНОСТЬ ЛИЧНОСТИ В СОВРЕМЕНН..."
36231,27239519,4,387234.0,"[['903'], 'none']",Молодов?,О.Б.,['Алтайский государственный педагогический уни...,Molodov?,O.B.,none,...,4,2,0.0,34330161,none,1,1,http://elibrary.ru/item.asp?id=27239519,none,"Молодов?, О.Б. Предпосылки регионального сепар..."
61975,37418742,4,0.0,"[['258'], ['258']]",М?рат?ызы,А.,['Алтайский государственный университет'],M?rat?yzy,A.,['Altai State University'],...,4,0,0.0,37418727,none,1,0,http://elibrary.ru/item.asp?id=37418742,none,"М?рат?ызы, А. СОВРЕМЕННОЕ СОСТОЯНИЕ МИГРАЦИОНН..."
66247,45784958,4,342670.0,"[['199'], 'none']",К?рчансен,М.,['Воронежский государственный университет'],K?rchansen,M.,none,...,4,0,0.0,45784951,none,1,1,http://elibrary.ru/item.asp?id=45784958,none,"К?рчансен, М. КУЛЬТУРНЫЙ НАЦИОНАЛИЗМ В НАЦИОНА..."


In [177]:
problem2.to_excel('problem2.xlsx', index=None)

И, в-третьих, обнаружилось что иногда в фамилиях оказываются вбиты имена (судя по отсутствующим инициалам). Это в основном случаи авторов с не русскими фамилиями ии имиенами (например, `Нгуен Тхе Зунг`, `Соларес Сиерра`), но сложно выделить единую логику по которым можно было бы определить что является инициалами, а что фамилией, поэтому нами было принято решение выгрузить такие строки отдельно и вручную пробежаться по этим авторам на сайте Elibrary. Встречаемость проблемы – 0.14% (в абсолютном значении - 108 случаев).

In [185]:
hard_names = []
problem3_ids = []
index = 0
for i in range(1, 8+1):
    for j in range(df.shape[0]):
        last_name = df.loc[j, f'author_{i}_ru last name']
        if len(last_name.split()) > 1 and '(' not in last_name:
            problem3_ids.append(df.loc[j, 'id'])

problem3_ids = list(set(problem3_ids))
# Решение - оценить масштаб проблемы (в публикациях)

In [186]:
print(len(problem3_ids))

108


In [188]:
problem3 = df[df['id'].isin(problem3_ids)]
problem3

,id,genreId,author_1_id,author_1_affiliation_id,author_1_ru last name,author_1_ru initials,author_1_ru affiliation,author_1_eng last name,author_1_eng initials,author_1_eng affiliation,...,genre_id,cited,corecited,parent_id,source_id,isFT,isNEW,link,url,ref
12,15515293,4,0.0,"[['15117'], 'none']",Нгуен Тхе Зунг,none,['МГУКИ'],Nguen the zung,none,none,...,4,0,0.0,33652048,none,1,1,http://elibrary.ru/item.asp?id=15515293,none,Нгуен Тхе Зунг СИСТЕМА ОБСЛУЖИВАНИЯ ЧИТАТЕЛЕЙ ...
52,15563443,4,0.0,"[['15117'], 'none']",Кан Ен Э,none,['МГУКИ'],Kan en e,none,none,...,4,0,0.0,33654722,none,1,1,http://elibrary.ru/item.asp?id=15563443,none,Кан Ен Э ХРИСТИАНСКАЯ ПЕВЧЕСКАЯ КУЛЬТУРА В ЛИТ...
71,15583296,4,249783.0,"[['500'], ['500']]",Аймаутова,Н.Е.,['Российский университет дружбы народов'],Ajmautova,N.E.,['Peoples Friendship University Russia'],...,4,0,0.0,33655525,none,1,0,http://elibrary.ru/item.asp?id=15583296,none,"Аймаутова, Н.Е. Социолог как продавец услуг. П..."
169,16558268,4,0.0,"[['867'], 'none']",Лань Цзюн Ли,Ц.Л.,['ТОГУ'],Lan' tszjun li,TS.L.,none,...,4,2,0.0,33666699,none,1,0,http://elibrary.ru/item.asp?id=16558268,none,"Лань Цзюн Ли, Ц.Л. СОВРЕМЕННЫЕ ПОДХОДЫ К ИССЛЕ..."
268,15211711,4,0.0,"[['15117'], 'none']",Сун Гуйцзюань,none,['МГУКИ'],Sun gujtszjuan',none,none,...,4,0,0.0,33600246,none,1,1,http://elibrary.ru/item.asp?id=15211711,none,"Сун Гуйцзюань ЭПОС ""ГЭСЭРИАДА"" И ЕГО КУЛЬТУРНЫ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70478,46636738,4,0.0,"['none', 'none']",фон Визе,Л.,none,Fon vize,L.,none,...,4,0,0.0,46636735,none,1,1,http://elibrary.ru/item.asp?id=46636738,none,"фон Визе, Л. Идеациональная и чувственная куль..."
72729,45608087,4,0.0,"[['2541'], ['2541']]",Тянь Сяоюй,none,['Московский государственный университет им. М...,Tjan' sjaojuj,none,['Lomonosov Moscow State University'],...,4,0,0.0,45608080,none,1,1,http://elibrary.ru/item.asp?id=45608087,none,Тянь Сяоюй Особенности регулирования социальны...
72985,43858676,4,0.0,"[['', '4812'], ['', '4812']]",Соларес Сиерра,Э.Б.,"['университет Ольгина', 'Российский государств...",Solares sierra,E.B.,"['University of Holgu?n', 'Russian State Socia...",...,4,0,0.0,43858673,none,1,1,http://elibrary.ru/item.asp?id=43858676,http://dom-hors.ru/rus/files/arhiv_zhurnala/sp...,"Соларес Сиерра, Э.Б. Молодые внутренние мигран..."
73132,35324794,4,0.0,"[['322'], 'none']",Майга Сори,И.,['Южный федеральный университет'],Majga sori,I.,none,...,4,0,0.0,35324772,none,1,0,http://elibrary.ru/item.asp?id=35324794,none,"Майга Сори, И. Способы урегулирования межэтнич..."


In [189]:
problem3.to_excel('problem3.xlsx', index=None)

## Часть 2.

In [ ]:
# На этом этапе предполагается, что до второй части вы вручную пробежались по проблемным строкам,
# можете их загрузить и обновить вашу таблицу, чтобы вместо старых "проблемных" строк остались новые
problem1_solved = pd.read_excel('problem1_solved.xlsx')
problem2_solved = pd.read_excel('problem2_solved.xlsx')
problem3_solved = pd.read_excel('problem3_solved.xlsx')
df = pd.concat([df, problem1_solved, problem2_solved, problem3_solved])
df = df.drop_duplicates(by='id', keep='last')